In [1]:
from Qommunity.searchers.hierarchical_community_searcher import HierarchicalCommunitySearcher
from Qommunity.samplers.hierarchical.gurobi_sampler import GurobiSampler

In [2]:
from Qommunity.samplers.regular.leiden_sampler import LeidenSampler
from Qommunity.searchers.community_searcher.community_searcher import CommunitySearcher

In [3]:
import numpy as np
import networkx as nx


adj_matrix = np.genfromtxt("data/Edge_AAL90_Binary.csv", delimiter="	")
G = nx.from_numpy_matrix(adj_matrix)

In [4]:
searcher = CommunitySearcher(LeidenSampler(G))

In [5]:
res_leiden = searcher.community_search()

In [6]:
res_leiden

[[2, 3, 4, 22, 23, 24, 25, 26, 27, 30, 31, 34, 35, 64, 65, 66, 67, 84, 85, 89],
 [1, 16, 17, 19, 28, 29, 32, 33, 56, 57, 62, 63, 68, 69, 78, 79, 80, 81, 83],
 [0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 60, 61, 88],
 [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 58, 59],
 [20, 21, 36, 37, 38, 39, 40, 41, 70, 71, 72, 73, 74, 75, 76, 77],
 [82, 86, 87]]

In [7]:
hierarchical_searcher = HierarchicalCommunitySearcher(GurobiSampler(G))

In [8]:
res_gurobi = hierarchical_searcher.hierarchical_community_search()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-27


In [9]:
res_gurobi

[[34, 35, 66, 67],
 [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
 [20, 21, 36, 37, 38, 39, 40, 41, 70, 71, 72, 73, 74, 75, 76, 77, 86, 87],
 [0,
  1,
  16,
  17,
  19,
  28,
  29,
  32,
  33,
  56,
  57,
  58,
  59,
  62,
  63,
  68,
  69,
  78,
  79,
  80,
  81,
  82,
  83],
 [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 60, 61, 88],
 [2, 3, 4, 22, 23, 24, 25, 26, 27, 30, 31, 64, 65, 84, 85, 89]]

In [10]:
nx.community.modularity(G, res_leiden)

0.646686912817234

In [11]:
nx.community.modularity(G, res_gurobi)

0.637922460795283

In [35]:
from samplers.hierarchical.advantage_sampler.advantage_sampler import AdvantageSampler


hierarchical_searcher = HierarchicalCommunitySearcher(AdvantageSampler(G, num_reads=100))

In [13]:
res_adv = hierarchical_searcher.hierarchical_community_search()

In [14]:
nx.community.modularity(G, res_adv)

0.6379224607952829

In [36]:
samples_reg = [hierarchical_searcher.hierarchical_community_search() for _ in range(10)]

In [39]:
mods_reg = [nx.community.modularity(G, res) for res in samples_reg]
mods_reg

[0.6379224607952829,
 0.6379224607952829,
 0.6434635125295256,
 0.6379224607952829,
 0.6379224607952829,
 0.6379224607952829,
 0.6379224607952829,
 0.6379224607952829,
 0.6396214798633681,
 0.639621479863368]

In [34]:
from minorminer.busclique import busgraph_cache

busgraph_cache.clear_all_caches()

In [18]:
from samplers.hierarchical.advantage_sampler_clique_embedding import AdvantageSamplerCliqueEmbedding


hierarchical_searcher_cl = HierarchicalCommunitySearcher(AdvantageSamplerCliqueEmbedding(G, num_reads=100))

comm: 90 elapsed: 142.30161905288696


In [19]:
res_adv_cl = hierarchical_searcher_cl.hierarchical_community_search()

comm: 90 elapsed: 0.10926389694213867
comm: 54 elapsed: 0.5647273063659668
comm: 31 elapsed: 0.1217963695526123
comm: 16 elapsed: 0.1115109920501709
comm: 15 elapsed: 0.08745813369750977
comm: 23 elapsed: 0.11761879920959473
comm: 36 elapsed: 0.09084701538085938
comm: 18 elapsed: 0.10961198806762695
comm: 4 elapsed: 0.09019279479980469
comm: 14 elapsed: 0.0916588306427002
comm: 18 elapsed: 0.09757471084594727


In [20]:
res_adv_cl

[[2, 3, 4, 22, 23, 24, 25, 26, 27, 30, 31, 64, 65, 84, 85, 89],
 [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 60, 61, 88],
 [0,
  1,
  16,
  17,
  19,
  28,
  29,
  32,
  33,
  56,
  57,
  58,
  59,
  62,
  63,
  68,
  69,
  78,
  79,
  80,
  81,
  82,
  83],
 [34, 35, 66, 67],
 [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55],
 [20, 21, 36, 37, 38, 39, 40, 41, 70, 71, 72, 73, 74, 75, 76, 77, 86, 87]]

In [21]:
nx.community.modularity(G, res_adv_cl)

0.6379224607952829

In [22]:
samples = [hierarchical_searcher_cl.hierarchical_community_search() for _ in range(10)]

comm: 90 elapsed: 0.09931421279907227
comm: 54 elapsed: 0.2613859176635742
comm: 31 elapsed: 0.20003676414489746
comm: 16 elapsed: 0.09751033782958984
comm: 15 elapsed: 0.12792205810546875
comm: 23 elapsed: 0.1225590705871582
comm: 36 elapsed: 0.10801386833190918
comm: 18 elapsed: 0.09670639038085938
comm: 4 elapsed: 0.14651918411254883
comm: 14 elapsed: 0.10071253776550293
comm: 18 elapsed: 0.13571858406066895
comm: 90 elapsed: 0.11251306533813477
comm: 54 elapsed: 0.12963032722473145
comm: 31 elapsed: 0.10170197486877441
comm: 16 elapsed: 0.10663914680480957
comm: 15 elapsed: 0.1046910285949707
comm: 23 elapsed: 0.10436439514160156
comm: 36 elapsed: 0.13609910011291504
comm: 18 elapsed: 0.14765262603759766
comm: 4 elapsed: 0.13188886642456055
comm: 14 elapsed: 0.11051249504089355
comm: 18 elapsed: 0.1045527458190918
comm: 90 elapsed: 0.1804337501525879
comm: 54 elapsed: 0.13768482208251953
comm: 31 elapsed: 0.10774827003479004
comm: 16 elapsed: 0.11724710464477539
comm: 15 elapsed: 0

In [23]:
modularities = [nx.community.modularity(G, res) for res in samples]
modularities

[0.6379224607952829,
 0.6379224607952829,
 0.6379224607952829,
 0.6379224607952829,
 0.6379224607952829,
 0.6379224607952829,
 0.6379224607952829,
 0.6379224607952829,
 0.6379224607952829,
 0.6379224607952829]